Today, we're going to take a close look at how numbers work in PostgreSQL – specifically, we're going to focus on those small details that make a huge difference.


We're going to be using `DB Fiddle` for this course. 

Navigate to: https://www.db-fiddle.com/

In the top right corner of the webpage, be sure to select `Database: PostgreSQL 13`

Now, in the `Schema SQL` pane on the left copy and paste the following, or just click this link here:https://www.db-fiddle.com/f/djHaTH4rHY7HTRaevY9xyU/0


```
CREATE TABLE IF NOT EXISTS "character" (
    "id" INT,
    "player_id" INT,
    "name" TEXT,
    "level" INT,
    "class" TEXT,
    "account_balance" NUMERIC(6, 2),
    "hp" INT,
    "mp" INT,
    "strength" INT,
    "wisdom" INT,
    "stat_modifier" INT,
    "weight" NUMERIC(9, 6),
    "height" NUMERIC(9, 6)
);
INSERT INTO "character" VALUES
    (1,1,'Kav',3,'wizard',899.34,100,200,12,20,1,65,1.72),
    (2,2,'Gniok',5,'warrior',423.83,170,80,20,23,4,101.4,2),
    (3,1,'Mnah',2,'warrior',0,140,60,16,20,-1,80.300003,1.9299999),
    (4,2,'Balryll',1,'warrior',723.12,130,50,15,0,2,78.900002,1.87),
    (5,3,'Mnadjaid',13,'wizard',4321.93,171,37,86,-3,61,1.67,null),
    (6,4,'Duroso',24,'wizard',7231.32,240,890,45,104,17,0.2,1),
    (7,5,'Rolamar',3,'warrior',753.42,150,65,18,21,-29,91.300003,1.85),
    (8,5,'Avealath',7,'warrior',1023,190,95,28,29,17,95.199997,1.9);


CREATE TABLE IF NOT EXISTS "player" (
    "id" INT,
    "first_name" TEXT,
    "last_name" TEXT
);
INSERT INTO "player" VALUES
    (1,'Alan','Gilman'),
    (2,'Kate','Friss'),
    (3,'Anne','Birch'),
    (4,'Adrian','Regos'),
    (5,'Stephen','Colvin');

```


Before we continue, let's review what we already know about numbers. Integers are expressed in the following way: 1, 2, 43, -27, etc. 

Fractions are expressed with a dot (.): 12.45, -401.238, etc. Please note that in some languages, you use a comma instead of a dot, but not in SQL! For example, 12,45 is incorrect.

In PostgreSQL (and in most programming languages) there are three kinds of numbers:

- Integer (integer): stores integers (e.g., -5, 0, 5, 42).

- Floating-point (real, double precision): stores fractions by approximating them in binary.

- Decimal (numeric): stores fractions in decimal format.


In this part, we'll discuss some of the differences between PostgreSQL's number data types.

Of course, you can use the four basic mathematical operations (add +, subtract -, multiply *, and divide /) with actual numbers in PostgreSQL. You can also use columns and constant numbers together. Take a look at the example below:

```
SELECT hp / 4 AS quarter_hp
FROM character;
```

## Exercise

For each character, display its name, level, and the sum of its hp and mp as the hmp column.




```
SELECT
  name,
  level,
  hp + mp AS hmp
FROM character;
```

# Numeric functions: concatenation


You can also join numbers and text values using the concatenation operator ||. 

Take a look:

```
SELECT 'Your character is at level ' || level AS level_info
FROM character;
```

Simple, right?

### Exercise

For each character above level 1, show the following text:

The account balance for NAME is MONEY.

Where NAME is the name of the character and MONEY is the current account balance. Name the column account_info.

```
SELECT 'The account balance for ' || name || ' is ' || account_balance || '.' AS account_info
FROM character
WHERE level > 1;
```

# Problems with subtraction

In PostgreSQL, simple operations can get very tricky. Let's discuss some of them, starting with subtraction.

### Exercise

For a character named 'Mnah', select the name, weight, height, and the result of the following calculation:

`weight - height - weight + height AS zero`

It should equal 0, right?



```
SELECT
  name,
  weight,
  height,
  weight - height - weight + height AS zero
FROM character
WHERE name = 'Mnah';
```

# Binary arithmetic is not exact

Oops... Something went wrong. Why did we get such an odd result?

The weight and height columns have a real data type, which is a floating point number. 

The internal representation is binary arithmetic, which means not all decimals can be represented precisely. 

Therefore, any computations are not performed exactly, as we can see.

What can be done about it? Nothing! That's the way binary arithmetic works.

If you need exact computations, use decimal, numeric and money (for currency) data types. When dealing with money values, always use decimal, numeric or money data types. 

Floating point numbers are for "scientific" computations on various measurements, like weight and height.

# Problems with division


Now you know that subtraction can be tricky. What about division?

How much is 1 / 4? Let's check that.

`SELECT 1 / 4 AS result;`


# Integer division

Wait... what? We got a 0? After being taught it's 0.25? 

Has someone deceived us?

Not really.

What happened here is called integer division. 

This occurs when both the dividend and the divisor are integers. Since they are integers, PostgreSQL wants to return an integer result to match the operand types. 

In other words, it brutally cuts off the decimal part, which is .25 in our case. The zero (0) is the only thing left.

So, how can we make sure that the result includes the decimal part? One way is to change the type of one of the values to a decimal (e.g., 1 / 4.0 instead of 1 / 4). 

In this course, we will always change the type of the denominator.

## Exercise

Run another example query to see what happens when you use a decimal number.

`SELECT 1 / 4.0 AS result;`

# Casting values to other types

Hah! 

It worked this time, didn't it? That's how you can force the desired result in PostgreSQL.

There is one problem with this method: What if both numbers are given as columns, like hp / level? We need to use another trick: explicitly converting one column to another data type. This procedure is called casting, and it uses the structure shown below:

```
SELECT (hp::numeric) / level AS result
FROM character;
```

The structure column::type changes the column to the specified type. You can do this also by using the CAST() function:

```
SELECT CAST(hp AS numeric) / level AS result
FROM character;
```

This function takes the name of the column to convert, the keyword AS, and the new data type.

In PostgreSQL, there are three kinds of number data types:

Integer data types with names like smallint, integer, bigint, etc.
Exact number data types (i.e., decimal types) with names like numeric, decimal, money.
Inexact number data types with names like real, double precision.
When dividing two integers in this course, we always cast the numerator to a numeric data type. In a real world application, you may need a different precision and a different casting (e.g., casting both numbers to real).

## Exercise

For each character, show its name, level, and the health points divided by the mana points as the ratio column.

Cast hp to the numeric data type to get a precise result.

```
SELECT
  name,
  level,
  (hp::numeric) / mp AS ratio
FROM character;
```

# Division by zero, part one

Okay, let's discuss the division problem further. As you know, there is one thing that you can never do: divide by zero. 

Let's see what happens when we follow the path of mathematical darkness and divide by zero...

`SELECT 5 / 0 AS result;`

# Division by zero, part two

Now let's see what happens if you have many rows, of which only one contains division by 0.

## Exercise

Warriors get their first wisdom points at Level 2 and we have one warrior at Level 1 with 0 Wisdom points. 

Let's see what happens.

```
SELECT
  name,
  mp::numeric / wisdom AS ratio
FROM character;
```

# Getting rid of division by zero

Oops! 

As you can see, the error occurs even if there is only a single row with a zero value in the denominator. 

How can we deal with this? 

There are a few ways to solve the problem. For now, you can use the good old WHERE clause to filter out any rows that contain a zero:

`WHERE column_name != 0`

We'll discuss other methods of dealing with division by zero later in our course.

## Exercise

In the template you can find the query from the previous exercise. Filter out rows with 0 wisdom points.

```
SELECT
  name,
  mp::numeric / wisdom AS ratio
FROM character
WHERE wisdom != 0;
```

# The modulo operator (%)

There's one more useful operator related to division: the modulo operator (%). 

It returns the remainder of one number divided by another number, like this:

`x % y`

For instance, 9 % 7 will return 2, because 9 / 7 is 1 with 2 as the remainder. 

This operator works with integer and decimal numbers.

## Exercise

In our game, you can increase your strength by 1 if you sacrifice 7 HP. 

For each character, show its name and calculate how many health points will be left if the player decides to sacrifice the maximum number of HP. Name the column hp_left.

```
SELECT
  name,
  hp % 7 AS hp_left
FROM character;
```

# The ROUND() function


Now you know how to subtract and divide in PostgreSQL, let's discuss other numeric functions. 

One of them is `ROUND(x)`. 

This function will round the number within parentheses to the nearest integer number. This is standard mathematical rounding: any decimal part equal to or greater than 0.5 will be rounded up.

```
SELECT ROUND(account_balance)
FROM character
WHERE id = 1;
```

The above query will take the account_balance of the character with ID of 1 (which is 899.34) and round it to the nearest integer (to 899). Remember that ROUND() won't change the type of the value returned.

## Exercise

For each character, show its name, its actual account_balance, and their account balance rounded to the nearest integer as the integer_balance column. Notice how rounding is applied.



```
SELECT
  name,
  account_balance,
  ROUND(account_balance, 0) AS integer_balance
FROM character;
```

# Precision in ROUND()


There is also another version of round, which takes two arguments: `ROUND(x, precision)`. 

The second argument is new and specifies the number of decimal places to be returned. For example,

`SELECT ROUND(136.123, 2);`

will return 136.12. Let's try it out.

# Exercise

Show each character's name, its actual account_balance, and the account balance rounded to a single decimal place as the rounded_balance column.

```
SELECT
  name,
  account_balance,
  ROUND(account_balance, 1) AS rounded_balance
FROM character;
```

# ROUND() in PostgreSQL – explanation


Before we move on, let's discuss the data types that work with `ROUND()` in PostgreSQL:

The single-argument function ROUND(x) accepts both numeric and double precision (floating-point number) data types.

If you want to round a number with precision to a specific decimal place with the two-argument ROUND(x, precision) function, the number being rounded must be either a numeric or a decimal type.

To round a floating point number and specify a precision at the same time, you have to first cast it as numeric. For example:

```
SELECT ROUND(weight::numeric, 2) AS rounded_weight
FROM character;
```

will round the weight to two decimal places.

## Exercise
Round height to two decimal places and show it as the rounded_height column.

```
SELECT ROUND(height::numeric, 2) AS rounded_height
FROM character;
```

# Rounding up

Let's learn how to round in another way. 

Sometimes we want to round up. We can do it using the `CEIL(number)` function (or the CEILING(number) function which is an alias).

## Exercise

Show the character's name and actual weight, followed by the weight rounded up as the weight_rounded_up column.

```
SELECT
  name,
  weight,
  CEIL(weight) AS weight_rounded_up
FROM character;
```

# Rounding down


That was the `CEIL()` function. 

Sometimes we need to round down, and the `FLOOR(number)` function will come in handy.

## Exercise

Show each character's name, actual account_balance, and the account_balance rounded down as the balance_rounded_down column.

```
SELECT
  name,
  account_balance,
  FLOOR(account_balance) AS balance_rounded_down
FROM character;
```

# The TRUNC() function

Another function to practice is `TRUNC()`. It always rounds towards zero. As with `ROUND()`, there are two variants of `TRUNC()`:

The single-argument `TRUNC(x)` takes a number of type numeric or double precision.

The two-argument `TRUNC(x, p)` takes an additional integer p argument which defines the number of decimal places.

Take a look at the example:

```
SELECT
  name,
  TRUNC(weight)    AS weight_no_decimal_places,
  TRUNC(weight, 1) AS weight_one_decimal_place
FROM character;
```

But there's a catch! 

The same which was true for the `ROUND()` function. 

The query above won't work because you may define the number of decimal places only for the numeric types, so you should cast the value:

```
SELECT
  name,
  TRUNC(weight) AS weight_no_decimal_places,
  TRUNC(weight::numeric, 1) AS weight_one_decimal_place
FROM character;
```

### Exercise

Show the character's name together with its account_balance, and the account balance truncated to one decimal place as the truncated_balance.

```
SELECT
  name,
  account_balance,
  TRUNC(account_balance::numeric, 1) AS truncated_balance
FROM character;
```

# The ABS() function


Let's take a look at the ABS(x) function which returns the absolute value of x. 

Non-negative numbers will be expressed as they are, but negative ones will be expressed without the negative sign. Have a look:

`SELECT ABS(3), ABS(-3);`

The above query returns 3 twice. The absolute value of 3 is 3, as is the absolute value of -3.

## Exercise

For each character, show its actual stat_modifier value and its stat_modifier's absolute value as the absolute_stat_modifier column.



```
SELECT
  stat_modifier,
  ABS(stat_modifier) AS absolute_stat_modifier
FROM character;
```

## Summary

Now let's summarize what we've learned in this part:

- Computations using floating point numbers are not always exact. Use the decimal, numeric, or money data types for all money columns and whenever precision matters.

- Dividing two integers is integer division, which is not always accurate. Use CAST(column AS TYPE) or the :: operator to avoid it.

- Avoid division by zero.

We've learned some useful functions:

- The modulo operation (x % y) returns the remainder of x by y division.

- ROUND(x, p) rounds x to the nearest integer or to the specified number of decimal digits (p).

- CEILING(x) rounds up to the nearest integer value.

- FLOOR(x) rounds down to the nearest integer value.

- ABS(x) returns the absolute value of x.

- TRUNC(x, p) – removes decimal digits given in p from the number x; 

- TRUNC(x) removes all decimal digits.

- The :: operator and the CAST() function convert data types.

The queries we've written so far have been fairly simple. Let's test your knowledge with some more advanced exercises.

# Homework

# Exercise
Calculate BMI for every character in the character table. BMI is calculated in the following way:

$\text{BMI} = \frac{\text{weight in  kilograms}}{(\text{height in meters})^2}$
 

Round the result to integer. Show each character's name and their calculated BMI as the bmi column.

Note: height is measured in meters. Remember to change the type of the ROUND() function's first argument to numeric.

```
SELECT
  name,
  ROUND((weight/(height * height))::numeric, 0) AS bmi
FROM character;
```

# Exercise

A healing potion costs 50 gold coins. Calculate how many healing potions each character (show their name column) with an account balance of at least 100 can buy (name the column: potion_amount) and how much money that each character will have left after that purchase (name the column: change).

```
SELECT
  name,
  FLOOR(account_balance / 50) AS potion_amount,
  (account_balance % 50) AS change
FROM character
WHERE account_balance >= 100;
```

# Exercise

Each warrior at Level 3 or higher can perform a special attack whose damage is calculated as follows:

Character's strength added to one-fourth of its hp, and then multiplied by the absolute value of stat_modifier. Here's the equation to visualize this:

$(strength + {hp \over 4.0}) * |statmodifier|$

For characters who have this attack available, show the character's name and the amount of damage dealt by this attack (name the column: damage).



```
SELECT
  name,
  (strength + hp / 4.0) * ABS(stat_modifier) AS damage
FROM character
WHERE class = 'warrior'
  AND level > 2;
```

# Exercise

For players with ID of 1 or 5, show:

- First and last name joined by a single space as the player_name column.

- Their characters' level.

- Height rounded to one decimal place as the rounded_height column.

- Account balance rounded down to the nearest integer as the account_balance column.


```
SELECT
  first_name || ' ' || last_name AS player_name,
  level,
  ROUND(height::numeric, 1) AS rounded_height,
  FLOOR(account_balance) AS account_balance
FROM player p
JOIN character c
  ON c.player_id = p.id
WHERE player_id IN (1, 5);
```